Use KL Divergence loss on Knowledge Distillation Task. You can use any teacher and student model (prefer small models). You need to show that it works, and update README.md with proper logs


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install torchsummary
    from torchsummary import summary

from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torchvision

import os
import time
import math

In [2]:
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                      #  transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.RandomAffine(degrees=10, shear = 10),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

train = datasets.CIFAR10(root = './data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)

# Do we have CUDA drivers for us?
cuda = torch.cuda.is_available()
print ("Cuda Available?", cuda)

dataloader_args = dict(shuffle=True, batch_size=2048, num_workers=2, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# Dataloaders
train_loader = torch.utils.data.DataLoader(dataset=train, **dataloader_args)
test_loader = torch.utils.data.DataLoader(dataset=test, **dataloader_args)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:05<00:00, 29.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Cuda Available? True


In [5]:
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.conv01 = nn.Conv2d(3, 16, 3, bias=False, padding=1)
        self.batch01 = nn.BatchNorm2d(num_features=16)

        # ---- Lets take a skip connection
        self.skip_conv1 = nn.Conv2d(16, 16, 3, padding=0, dilation=2)

        self.conv02 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch02 = nn.BatchNorm2d(num_features=16)
        self.conv03 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch03 = nn.BatchNorm2d(num_features=16)
        self.conv04 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch04 = nn.BatchNorm2d(num_features=16)
        self.pool01 = nn.MaxPool2d(2, 2)                                #O=16
        self.conv05 = nn.Conv2d(16, 16, 1, bias=False)

        self.conv11 = nn.Conv2d(16, 64, 3, bias=False, padding=1)
        self.batch11 = nn.BatchNorm2d(num_features=64)
        self.conv12 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch12 = nn.BatchNorm2d(num_features=64)
        self.conv13 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch13 = nn.BatchNorm2d(num_features=64)
        self.conv14 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch14 = nn.BatchNorm2d(num_features=64)
        self.pool11 = nn.MaxPool2d(2, 2)                                #O=8
        self.conv15 = nn.Conv2d(64, 64, 1, bias=False)

        self.conv21 = nn.Conv2d(64, 128, 3, bias=False, padding=1)
        self.batch21 = nn.BatchNorm2d(num_features=128)
        self.conv22 = nn.Conv2d(128, 128, 3, bias=False, padding=1)
        self.batch22 = nn.BatchNorm2d(num_features=128)
        self.conv23 = nn.Conv2d(128,128, 3, bias=False, padding=1)
        self.batch23 = nn.BatchNorm2d(num_features=128)
        self.conv24 = nn.Conv2d(128, 128, 3, bias=False, padding=1)
        self.batch24 = nn.BatchNorm2d(num_features=128)
        self.pool21 = nn.MaxPool2d(2, 2)                                #O=4
        self.conv25 = nn.Conv2d(128, 128, 1, bias=False)

        self.conv31 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, groups=128, bias = False, padding = 1)
        self.convPV1= nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, bias = False, padding = 0)
        self.batch31 = nn.BatchNorm2d(num_features=128)
        self.conv32 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, groups=128, bias = False, padding = 1)
        self.convPV2= nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, bias = False, padding = 0)
        self.batch32 = nn.BatchNorm2d(num_features=256)


        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.convx3 = nn.Conv2d(256, 10, 1, bias=False, padding=0)

    def forward(self, x):
        x = self.batch01(F.relu(self.conv01(x)))

        # ---- Lets take a skip connection
        skip_channels = self.skip_conv1(self.skip_conv1(self.skip_conv1(self.skip_conv1(x))))

        x = self.batch02(F.relu(self.conv02(x)))
        x = self.batch03(F.relu(self.conv03(x)))
        x = self.batch04(F.relu(self.conv04(x)))
        x = self.pool01(x)
        x = self.conv05(x)
        # ----------------------------------------------------------

        # ---- Lets add the skip connection here
        x = skip_channels + x

        x = self.batch11(F.relu(self.conv11(x)))
        x = self.batch12(F.relu(self.conv12(x)))
        x = self.batch13(F.relu(self.conv13(x)))
        x = self.batch14(F.relu(self.conv14(x)))
        x = self.pool11(x)
        x = self.conv15(x)
        # ----------------------------------------------------------

        x = self.batch21(F.relu(self.conv21(x)))
        x = self.batch22(F.relu(self.conv22(x)))
        x = self.batch23(F.relu(self.conv23(x)))
        x = self.batch24(F.relu(self.conv24(x)))
        x = self.pool21(x)
        x = self.conv25(x)
        # ----------------------------------------------------------

        x = self.batch31(F.relu(self.convPV1(F.relu(self.conv31(x)))))
        x = self.batch32(F.relu(self.convPV2(F.relu(self.conv32(x)))))


        x = self.avg_pool(x)
        x = self.convx3(x)
        x = x.view(-1, 10)                           # Don't want 10x1x1..
        return F.log_softmax(x, dim=1)  # Added dim=1 parameter)

In [6]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_taken = []

class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.prev_loss = None  # Initialize as None
        self.counter = 0

    def __call__(self, train_loss):
        if self.prev_loss is None:  # First iteration
            self.prev_loss = train_loss
            return False  # Continue training

        if (abs(train_loss - self.prev_loss)) < self.min_delta:
            print(f'---------- prev = {self.prev_loss} current = {train_loss} ---------')
            self.counter += 1
        else:
            self.counter = 0  # Reset counter if loss improves

        self.prev_loss = train_loss

        return self.counter >= self.tolerance  # Return True if stopping criteria met



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)

    correct = 0
    processed = 0
    epoch_loss = 0
    time_taken.clear()

    for batch_idx, (data, target) in enumerate(pbar):
        t0 = time.time()

        data, target = data.to(device), target.to(device)

        # Don't want history of gradients
        optimizer.zero_grad()

        y_predict = model(data)

        # Calculate loss
        loss = F.nll_loss(y_predict, target)
        epoch_loss += loss.item()

        # Backpropagate error
        loss.backward()

        # Take an optimizer step
        optimizer.step()

        torch.cuda.synchronize()
        t1 = time.time()

        time_taken.append((t1 - t0))

        pred = y_predict.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    return avg_train_loss


def test(model, device, test_loader):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss

In [7]:
# Initialize model, optimizer, and early stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (f'Device Using = {device}')
model = TeacherModel().to(device)
summary(model, input_size=(3, 32, 32))
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
early_stopping = EarlyStopping(tolerance=5, min_delta=0.02)

EPOCHS = 100
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    avg_train_loss = train(model, device, train_loader, optimizer, epoch)
    print(f" --> EPOCH: {epoch}, Avg Training Loss: {avg_train_loss:.4f}, Avg Time Taken = {(sum(time_taken) / len(time_taken)) * 1000:.2f}ms")
    val_loss = test(model, device, test_loader)

    # Check for early stopping
    if early_stopping(avg_train_loss):
        try:
            # Ensure the directory exists
            save_dir = '/content/drive/MyDrive/EPAi_V5'
            os.makedirs(save_dir, exist_ok=True)
            PATH = os.path.join(save_dir, f'model_heavy_acc_{int(train_acc[-1]):d}.pth')
        except OSError:
            # Fallback to current directory if Drive is unavailable
            PATH = f'./model_heavy_acc_{int(train_acc[-1]):d}.pth'

        # Save the model weights
        torch.save(model.state_dict(), PATH)
        print(f"Model saved at: {PATH}")
        print("Early stopping triggered!")
        break

Device Using = cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
            Conv2d-4           [-1, 16, 24, 24]           2,320
            Conv2d-5           [-1, 16, 20, 20]           2,320
            Conv2d-6           [-1, 16, 16, 16]           2,320
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
            Conv2d-9           [-1, 16, 32, 32]           2,304
      BatchNorm2d-10           [-1, 16, 32, 32]              32
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
        MaxPool2d-13           [-1, 16, 16, 16]               0
           Conv2d-1

Loss=1.7569657564163208 Batch_id=24 Accuracy=24.23: 100%|██████████| 25/25 [00:18<00:00,  1.34it/s]

 --> EPOCH: 0, Avg Training Loss: 2.0317, Avg Time Taken = 428.80ms



Test set: Average loss: 2.3616, Accuracy: 1000/10000 (10.00%)

Epoch 2/100


Loss=1.489882469177246 Batch_id=24 Accuracy=42.15: 100%|██████████| 25/25 [00:17<00:00,  1.42it/s]

...

 --> EPOCH: 28, Avg Training Loss: 0.2634, Avg Time Taken = 467.57ms



Test set: Average loss: 0.7115, Accuracy: 7846/10000 (78.46%)

---------- prev = 0.27017143309116365 current = 0.26343166947364804 ---------
Epoch 30/100


Loss=0.3437854051589966 Batch_id=24 Accuracy=91.23: 100%|██████████| 25/25 [00:18<00:00,  1.32it/s]

 --> EPOCH: 29, Avg Training Loss: 0.2569, Avg Time Taken = 465.19ms



Test set: Average loss: 0.7036, Accuracy: 7891/10000 (78.91%)

---------- prev = 0.26343166947364804 current = 0.2568626445531845 ---------
Model saved at: /content/drive/MyDrive/EPAi_V5/model_heavy_acc_91.pth
Early stopping triggered!


In [8]:
model.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_heavy_acc_91.pth', weights_only=True))
test(model, device, test_loader)


Test set: Average loss: 0.7036, Accuracy: 7891/10000 (78.91%)



0.7035769897460937

In [9]:
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.conv01 = nn.Conv2d(3, 16, 3, bias=False, padding=1)
        self.batch01 = nn.BatchNorm2d(num_features=16)

        # ---- Lets take a skip connection
        self.skip_conv1 = nn.Conv2d(16, 16, 3, padding=0, dilation=2)

        self.conv02 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch02 = nn.BatchNorm2d(num_features=16)
        self.conv03 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch03 = nn.BatchNorm2d(num_features=16)
        self.conv04 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch04 = nn.BatchNorm2d(num_features=16)
        self.pool01 = nn.MaxPool2d(2, 2)                                #O=16
        self.conv05 = nn.Conv2d(16, 16, 1, bias=False)

        self.conv11 = nn.Conv2d(16, 32, 3, bias=False, padding=1)
        self.batch11 = nn.BatchNorm2d(num_features=32)
        self.conv12 = nn.Conv2d(32, 32, 3, bias=False, padding=1)
        self.batch12 = nn.BatchNorm2d(num_features=32)
        self.conv13 = nn.Conv2d(32, 32, 3, bias=False, padding=1)
        self.batch13 = nn.BatchNorm2d(num_features=32)
        self.conv14 = nn.Conv2d(32, 32, 3, bias=False, padding=1)
        self.batch14 = nn.BatchNorm2d(num_features=32)
        self.pool11 = nn.MaxPool2d(2, 2)                                #O=8
        self.conv15 = nn.Conv2d(32, 32, 1, bias=False)

        self.conv21 = nn.Conv2d(32, 64, 3, bias=False, padding=1)
        self.batch21 = nn.BatchNorm2d(num_features=64)
        self.conv22 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch22 = nn.BatchNorm2d(num_features=64)
        self.conv23 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch23 = nn.BatchNorm2d(num_features=64)
        self.conv24 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch24 = nn.BatchNorm2d(num_features=64)
        self.pool21 = nn.MaxPool2d(2, 2)                                #O=4
        self.conv25 = nn.Conv2d(64, 64, 1, bias=False)

        self.conv31 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, groups=64, bias = False, padding = 1)
        self.convPV1= nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, bias = False, padding = 0)
        self.batch31 = nn.BatchNorm2d(num_features=128)
        self.conv32 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, groups=128, bias = False, padding = 1)
        self.convPV2= nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, bias = False, padding = 0)
        self.batch32 = nn.BatchNorm2d(num_features=256)


        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.convx3 = nn.Conv2d(256, 10, 1, bias=False, padding=0)

    def forward(self, x):
        x = self.batch01(F.relu(self.conv01(x)))

        # ---- Lets take a skip connection
        skip_channels = self.skip_conv1(self.skip_conv1(self.skip_conv1(self.skip_conv1(x))))

        x = self.batch02(F.relu(self.conv02(x)))
        x = self.batch03(F.relu(self.conv03(x)))
        x = self.batch04(F.relu(self.conv04(x)))
        x = self.pool01(x)
        x = self.conv05(x)
        # ----------------------------------------------------------

        # ---- Lets add the skip connection here
        x = skip_channels + x

        x = self.batch11(F.relu(self.conv11(x)))
        x = self.batch12(F.relu(self.conv12(x)))
        x = self.batch13(F.relu(self.conv13(x)))
        x = self.batch14(F.relu(self.conv14(x)))
        x = self.pool11(x)
        x = self.conv15(x)
        # ----------------------------------------------------------

        x = self.batch21(F.relu(self.conv21(x)))
        x = self.batch22(F.relu(self.conv22(x)))
        x = self.batch23(F.relu(self.conv23(x)))
        x = self.batch24(F.relu(self.conv24(x)))
        x = self.pool21(x)
        x = self.conv25(x)
        # ----------------------------------------------------------

        x = self.batch31(F.relu(self.convPV1(F.relu(self.conv31(x)))))
        x = self.batch32(F.relu(self.convPV2(F.relu(self.conv32(x)))))


        x = self.avg_pool(x)
        x = self.convx3(x)
        x = x.view(-1, 10)                           # Don't want 10x1x1..
        return F.log_softmax(x, dim=1)  # Added dim=1 parameter)

In [10]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_taken = []

class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.prev_loss = None  # Initialize as None
        self.counter = 0

    def __call__(self, train_loss):
        if self.prev_loss is None:  # First iteration
            self.prev_loss = train_loss
            return False  # Continue training

        if (abs(train_loss - self.prev_loss)) < self.min_delta:
            print(f'---------- prev = {self.prev_loss} current = {train_loss} ---------')
            self.counter += 1
        else:
            self.counter = 0  # Reset counter if loss improves

        self.prev_loss = train_loss

        return self.counter >= self.tolerance  # Return True if stopping criteria met



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)

    correct = 0
    processed = 0
    epoch_loss = 0
    time_taken.clear()

    for batch_idx, (data, target) in enumerate(pbar):
        t0 = time.time()

        data, target = data.to(device), target.to(device)

        # Don't want history of gradients
        optimizer.zero_grad()

        y_predict = model(data)

        # Calculate loss
        loss = F.nll_loss(y_predict, target)
        epoch_loss += loss.item()

        # Backpropagate error
        loss.backward()

        # Take an optimizer step
        optimizer.step()

        torch.cuda.synchronize()
        t1 = time.time()

        time_taken.append((t1 - t0))

        pred = y_predict.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    return avg_train_loss


def test(model, device, test_loader):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss

In [11]:
# Initialize model, optimizer, and early stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (f'Device Using = {device}')
model = StudentModel().to(device)
summary(model, input_size=(3, 32, 32))
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
early_stopping = EarlyStopping(tolerance=5, min_delta=0.02)

EPOCHS = 100
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    avg_train_loss = train(model, device, train_loader, optimizer, epoch)
    print(f" --> EPOCH: {epoch}, Avg Training Loss: {avg_train_loss:.4f}, Avg Time Taken = {(sum(time_taken) / len(time_taken)) * 1000:.2f}ms")
    val_loss = test(model, device, test_loader)

    # Check for early stopping
    if early_stopping(avg_train_loss):
        try:
            # Ensure the directory exists
            save_dir = '/content/drive/MyDrive/EPAi_V5'
            os.makedirs(save_dir, exist_ok=True)
            PATH = os.path.join(save_dir, f'model_small_acc_{int(train_acc[-1]):d}.pth')
        except OSError:
            # Fallback to current directory if Drive is unavailable
            PATH = f'./model_small_acc_{int(train_acc[-1]):d}.pth'

        # Save the model weights
        torch.save(model.state_dict(), PATH)
        print(f"Model saved at: {PATH}")
        print("Early stopping triggered!")
        break

Device Using = cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
            Conv2d-4           [-1, 16, 24, 24]           2,320
            Conv2d-5           [-1, 16, 20, 20]           2,320
            Conv2d-6           [-1, 16, 16, 16]           2,320
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
            Conv2d-9           [-1, 16, 32, 32]           2,304
      BatchNorm2d-10           [-1, 16, 32, 32]              32
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
        MaxPool2d-13           [-1, 16, 16, 16]               0
           Conv2d-1

Loss=1.7643420696258545 Batch_id=24 Accuracy=22.57: 100%|██████████| 25/25 [00:17<00:00,  1.42it/s]

 --> EPOCH: 0, Avg Training Loss: 2.0624, Avg Time Taken = 357.53ms



Test set: Average loss: 2.3772, Accuracy: 1000/10000 (10.00%)

Epoch 2/100


Loss=1.5391430854797363 Batch_id=24 Accuracy=37.13: 100%|██████████| 25/25 [00:18<00:00,  1.37it/s]

...

Loss=0.5759264230728149 Batch_id=24 Accuracy=80.44: 100%|██████████| 25/25 [00:17<00:00,  1.44it/s]

 --> EPOCH: 25, Avg Training Loss: 0.5586, Avg Time Taken = 360.99ms



Test set: Average loss: 0.7487, Accuracy: 7449/10000 (74.49%)

---------- prev = 0.5649992895126342 current = 0.5586209177970887 ---------
Epoch 27/100


Loss=0.6129828691482544 Batch_id=24 Accuracy=80.87: 100%|██████████| 25/25 [00:18<00:00,  1.37it/s]


 --> EPOCH: 26, Avg Training Loss: 0.5443, Avg Time Taken = 361.21ms

Test set: Average loss: 0.7521, Accuracy: 7460/10000 (74.60%)

---------- prev = 0.5586209177970887 current = 0.5442671251296997 ---------
Model saved at: /content/drive/MyDrive/EPAi_V5/model_small_acc_80.pth
Early stopping triggered!


In [12]:
model.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_small_acc_80.pth', weights_only=True))
test(model, device, test_loader)


Test set: Average loss: 0.7521, Accuracy: 7460/10000 (74.60%)



0.7520527465820313

# Distillation Process

In [13]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.prev_loss = None  # Initialize as None
        self.counter = 0

    def __call__(self, train_loss):
        if self.prev_loss is None:  # First iteration
            self.prev_loss = train_loss
            return False  # Continue training

        if (abs(train_loss - self.prev_loss)) < self.min_delta:
            print(f'---------- prev = {self.prev_loss} current = {train_loss} ---------')
            self.counter += 1
        else:
            self.counter = 0  # Reset counter if loss improves

        self.prev_loss = train_loss

        return self.counter >= self.tolerance  # Return True if stopping criteria met

train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_taken = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device = {device}')

# Prepare teacher model
teacher = TeacherModel().to(device)
teacher.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_heavy_acc_92.pth', weights_only=True))
teacher.eval()

# Prepare student model
student = StudentModel().to(device)
student.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_small_acc_81.pth', weights_only=True))

print("=============================================")
print("Student model accuracy before training ")
student.eval()
test(student, device, test_loader)
print("=============================================")

# Loss functions
hard_loss = nn.CrossEntropyLoss() #Hard label loss
soft_loss = nn.KLDivLoss(reduction="batchmean")  # Distillation loss

# Temperature and alpha
T = 5.0  # Temperature
alpha = 0.5  # Weight for distillation loss

# Optimizer
optimizer = optim.Adam(student.parameters(), lr=0.001)

# Training loop
EPOCHS = 100
for epoch in range(EPOCHS):

    correct = 0
    processed = 0
    epoch_loss = 0
    pbar = tqdm(train_loader)
    student.train()
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # Teacher predictions (soft labels)
        with torch.no_grad():
            teacher_logits = teacher(data) / T

        teacher_probs = torch.softmax(teacher_logits, dim=1)

        # Student predictions
        student_logits = student(data)
        student_probs = torch.log_softmax(student_logits / T, dim=1)

        # Compute losses
        loss_soft = soft_loss(student_probs, teacher_probs) * (T ** 2)  # Scale by T^2
        loss_hard = hard_loss(student_logits, target)
        loss = alpha * loss_hard + (1 - alpha) * loss_soft

        epoch_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

        pred = student_logits.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Check for early stopping
    if early_stopping(avg_train_loss):
        try:
            # Ensure the directory exists
            save_dir = '/content/drive/MyDrive/EPAi_V5'
            os.makedirs(save_dir, exist_ok=True)
            PATH = os.path.join(save_dir, f'model_distil_acc_{int(train_acc[-1]):d}.pth')
        except OSError:
            # Fallback to current directory if Drive is unavailable
            PATH = f'./model_small_acc_{int(train_acc[-1]):d}.pth'

        # Save the model weights
        torch.save(model.state_dict(), PATH)
        print(f"Model saved at: {PATH}")
        print("Early stopping triggered!")
        break

    student.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = student(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



Using device = cuda
Student model accuracy before training 

Test set: Average loss: 0.7419, Accuracy: 7566/10000 (75.66%)



Loss=1.3082377910614014 Batch_id=24 Accuracy=76.71: 100%|██████████| 25/25 [00:18<00:00,  1.32it/s]



Test set: Average loss: 0.9304, Accuracy: 7478/10000 (74.78%)



Loss=1.2178696393966675 Batch_id=24 Accuracy=80.33: 100%|██████████| 25/25 [00:18<00:00,  1.37it/s]



Test set: Average loss: 0.7959, Accuracy: 7728/10000 (77.28%)



...

---------- prev = 0.8424052166938781 current = 0.8303352212905883 ---------



Test set: Average loss: 0.6974, Accuracy: 7936/10000 (79.36%)



Loss=0.8141599297523499 Batch_id=24 Accuracy=86.05: 100%|██████████| 25/25 [00:18<00:00,  1.37it/s]

---------- prev = 0.8303352212905883 current = 0.8178793621063233 ---------


IndexError: list index out of range